Basic Join 

The Report

In [ ]:
SELECT
    CASE
    WHEN grades.grade < 8 THEN 'NULL'
    ELSE students.name
    END,
    grades.grade, students.marks
FROM students, grades
WHERE students.marks >= grades.min_mark AND students.marks <= grades.max_mark
ORDER BY grades.grade DESC, students.name

Top Competitors

In [ ]:
SELECT h.hacker_id, h.name
FROM Hackers h
JOIN (
  SELECT s.hacker_id, COUNT(DISTINCT s.challenge_id) AS full_scores
  FROM Submissions s
  JOIN Challenges c ON s.challenge_id = c.challenge_id
  JOIN Difficulty d ON c.difficulty_level = d.difficulty_level AND s.score = d.score
  GROUP BY s.hacker_id
  HAVING COUNT(DISTINCT s.challenge_id) > 1
) AS full_scores ON h.hacker_id = full_scores.hacker_id
ORDER BY full_scores DESC, h.hacker_id ASC;

Ollivander's Inventory

In [ ]:
SELECT W.ID, P.AGE, W.COINS_NEEDED, W.POWER 
FROM WANDS AS W
JOIN WANDS_PROPERTY AS P
ON (W.CODE = P.CODE) 
WHERE P.IS_EVIL = 0 AND W.COINS_NEEDED = (SELECT MIN(W2.COINS_NEEDED) 
                                          FROM WANDS AS W2
                                          JOIN WANDS_PROPERTY AS Y 
                                          ON (W2.CODE = Y.CODE) 
                                          WHERE W2.POWER = W.POWER AND Y.AGE = P.AGE) 
ORDER BY W.POWER DESC, P.AGE DESC;

Draw The Triangle 1

In [ ]:
Declare @i int = 20

while (@i>0)
begin
    print replicate('* ',@i);
    set @i = @i- 1;
end

Challenges

In [ ]:
SELECT hacker_id, name, c_cnt        
FROM (
    SELECT hacker_id, name, c_cnt,
           COUNT(*) OVER(PARTITION BY c_cnt) AS same_cnt,    
           MAX(c_cnt) OVER() AS max_cnt                
    FROM (
        SELECT h.hacker_id, h.name, COUNT(*) AS c_cnt
        FROM Hackers h
        JOIN Challenges c ON h.hacker_id = c.hacker_id
        GROUP BY h.hacker_id, h.name
        ) t1
        ) t2
WHERE c_cnt = max_cnt              
OR  (c_cnt != max_cnt AND same_cnt = 1)
ORDER BY c_cnt DESC, hacker_id;   

Draw The Triangle 2

In [ ]:
Declare @i int = 1

while (@i<=20)
begin
    print replicate('* ',@i);
    set @i = @i+ 1;
end

Contest Leaderboard

In [ ]:
WITH max_scores
AS (
    select hacker_id, max(score) as max_score
    from submissions
    group by hacker_id, challenge_id
)
select ms.hacker_id, h.name, sum(ms.max_score) as total_score
from max_scores ms 
join hackers h 
    on ms.hacker_id = h.hacker_id
group by ms.hacker_id, h.name
having sum(ms.max_score) <> 0
ORDER BY total_score DESC, ms.hacker_id;